# Home Work

В этой работе будем иследовать одну из популярнейших рекомендательных моделей - **Latent Factor Model** - https://arxiv.org/pdf/1912.04754. 

Перед выполнением задания нужно убедиться, что прогоняется бейзлайн. Для этого:
1) Скачайте  файлы - **node2name.json** и **clickstream.parque** с необходимыми данными
2) Положите в репозиторий ноутбука и запустите код

В этой работе вам нужно:
1) перебрать параметры модели - edim,batch_size, lr, epoch , num_negatives -   (по **1 балу - 5 балов**) 
2) Тип OPTIMIZER_NAME - (**4 бала за 5 оптимизаторов**)
3) На основе имеющихся данных собрать лучшую модель (по **precision@30**) и рассчитать ее метрики (**4 бала**)
4) Попробовать другие модели (например  als - https://benfred.github.io/implicit/ , gru4rec, sasrec  ) - за sasrec на хорошем уровне сразу **10 балов**. За другие модели по **3 бала**
5) По окончанию работы в mlflow настроить графики для сравнения моделей. Можно проявить фантазию, но обязательно должно быть сравнение с бейзлайном (данный ноутбук) против других моделей
6) В mlflow залогировать последнюю версию ноутбука - необходимое условия. Либо в github, но тогда прикрепить ссылку в [mlflow](http://84.201.128.89:90/) . Эксперимент в формате - **homework-\<name\>**
7) Доп балы (**20 баллов**) тому у кого будет наибольший скор на тесте. Но ваш ноутбук должен прогонятся и быть вопроизводимым.

Суммарно за работу **20 балов**

In [112]:
import json

with open('data/node2name.json', 'r') as f:
    node2name = json.load(f)

node2name = {int(k):v for k,v in node2name.items()}


In [113]:
node2name


{1047504: 'root',
 1047789: 'root -> Транспорт',
 1047505: 'root -> Услуги',
 1047737: 'root -> Для дома и дачи',
 1054252: 'root -> Хобби и отдых',
 1054319: 'root -> Электроника',
 1054439: 'root -> Личные вещи',
 1057415: 'root -> Готовый бизнес и оборудование',
 1057433: 'root -> Работа',
 1054803: 'root -> Недвижимость',
 1047597: 'root -> Животные',
 1047506: 'root -> Услуги -> Предложения услуг',
 1047635: 'root -> Животные -> Аквариум',
 1047634: 'root -> Животные -> Птицы',
 1047645: 'root -> Животные -> Товары для животных',
 1047598: 'root -> Животные -> Кошки',
 1047646: 'root -> Животные -> Собаки',
 1047636: 'root -> Животные -> Другие животные',
 1047764: 'root -> Для дома и дачи -> Бытовая техника',
 1047761: 'root -> Для дома и дачи -> Посуда и товары для кухни',
 1047751: 'root -> Для дома и дачи -> Ремонт и строительство',
 1047749: 'root -> Для дома и дачи -> Растения',
 1047750: 'root -> Для дома и дачи -> Продукты питания',
 1047738: 'root -> Для дома и дачи -> Ме

In [114]:
import pandas as pd

df = pd.read_parquet('data/clickstream.parque')
df = df.head(100_000)


In [115]:
df['is_train'] = df['event_date']< df['event_date'].max() - pd.Timedelta('2 day')
df['names'] = df['node_id'].map(node2name)

In [116]:
df

,cookie_id,event_date,node_id,is_train,names
0,15157399,2024-02-21 11:20:01,1047840,True,root -> Транспорт -> Запчасти и аксессуары -> ...
1,15157399,2024-03-05 10:24:54,1047561,True,root -> Услуги -> Предложения услуг -> Красота...
2,15157399,2024-03-05 10:28:55,1047561,True,root -> Услуги -> Предложения услуг -> Красота...
3,15157399,2024-04-13 11:22:25,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...
4,15157399,2024-04-13 11:22:45,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...
...,...,...,...,...,...
99995,20324089388,2024-04-06 20:22:13,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."
99996,20324089388,2024-04-06 20:24:20,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."
99997,20324089388,2024-04-06 20:27:25,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."
99998,20324089388,2024-04-06 20:33:10,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."


In [117]:
train_cooks = df[df['is_train']]['cookie_id'].unique()
train_items = df[df['is_train']]['node_id'].unique()


df = df[(df['cookie_id'].isin(train_cooks)) & (df['node_id'].isin(train_items))]


In [118]:
user_indes, index2user_id = pd.factorize(df['cookie_id'])
df['user_index'] = user_indes

node_indes, index2node = pd.factorize(df['node_id'])
df['node_index'] = node_indes


C:\Users\olegf\AppData\Local\Temp\ipykernel_19916\3144963355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_index'] = user_indes
C:\Users\olegf\AppData\Local\Temp\ipykernel_19916\3144963355.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['node_index'] = node_indes


In [119]:
df['node_index'].max()

2175

In [120]:
df

,cookie_id,event_date,node_id,is_train,names,user_index,node_index
0,15157399,2024-02-21 11:20:01,1047840,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,0
1,15157399,2024-03-05 10:24:54,1047561,True,root -> Услуги -> Предложения услуг -> Красота...,0,1
2,15157399,2024-03-05 10:28:55,1047561,True,root -> Услуги -> Предложения услуг -> Красота...,0,1
3,15157399,2024-04-13 11:22:25,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,2
4,15157399,2024-04-13 11:22:45,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,2
...,...,...,...,...,...,...,...
99995,20324089388,2024-04-06 20:22:13,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа...",1663,212
99996,20324089388,2024-04-06 20:24:20,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа...",1663,212
99997,20324089388,2024-04-06 20:27:25,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа...",1663,212
99998,20324089388,2024-04-06 20:33:10,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа...",1663,212


In [121]:
df_train, df_test = df[df['is_train']], df[~df['is_train']]
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


df_train.shape, df_test.shape

((96611, 7), (3333, 7))

In [122]:
import torch
from torch import nn
import random 
from tqdm.auto import tqdm

from torch.utils.data import Dataset, DataLoader


class RecDataset(Dataset):
    def __init__(self, users, items, item_per_users):
        self.users = users
        self.items = items
        self.item_per_users=item_per_users

    def __len__(self):
        return len(self.users)

    def __getitem__(self, i):
        user = self.users[i]
        return torch.tensor(user), torch.tensor(self.items[i]), self.item_per_users[user]


class LatentFactorModel(nn.Module):
    def __init__(self, edim, user_indexes, node_indexes):
        super(LatentFactorModel, self).__init__()
        self.edim = edim
        self.users = nn.Embedding(max(user_indexes) + 1, edim)
        self.items = nn.Embedding(max(node_indexes) + 1, edim)

    def forward(self, users, items):
        user_embedings = self.users(users).reshape(-1, self.edim )
        item_embedings = self.items(items)
        res = torch.einsum('be,bne->bn', user_embedings, item_embedings)
        return res 

    def pred_top_k(self, users, K=10):
        user_embedings = self.users(users).reshape(-1, self.edim )
        item_embedings = self.items.weight
        res = torch.einsum('ue,ie->ui', user_embedings, item_embedings)
        return torch.topk(res, K, dim=1)

    


def collate_fn(batch, num_negatives, num_items):
    users, target_items, users_negatives = [],[], []
    for triplets in batch:
        user, target_item, seen_item = triplets
        
        users.append(user)
        target_items.append(target_item)
        user_negatives = []
        
        while len(user_negatives)< num_negatives:
            candidate = random.randint(0, num_items)
            if candidate not in seen_item:
                user_negatives.append(candidate)
                
        users_negatives.append(user_negatives)
                
    
    positive = torch.ones(len(batch), 1)       
    negatives = torch.zeros(len(batch), num_negatives)
    labels = torch.hstack([positive, negatives])
    # print(torch.tensor(target_items))
    # print(users_negatives)
    items = torch.hstack([torch.tensor(target_items).reshape(-1, 1), torch.tensor(users_negatives)])
    return torch.hstack(users), items, labels

    

In [123]:
user2seen = df_train.groupby('user_index')['node_index'].agg(lambda x: list(set(x)))

In [124]:
BATCH_SIZE = 50_000
NUM_NEGATIVES = 5
EDIM = 128
EPOCH = 10
OPTIMIZER_NAME = 'Adam'
LR = 1

train_dataset = RecDataset(df_train['user_index'].values, df_train['node_index'], user2seen)


dataloader = DataLoader(train_dataset, shuffle=True,num_workers=0, batch_size=BATCH_SIZE,collate_fn=lambda x: collate_fn(x, NUM_NEGATIVES, max(df['node_index'].values)))


model = LatentFactorModel(EDIM, user_indes, node_indes)
optimizer = torch.optim.Adam(model.parameters(), LR)
 
bar = tqdm(total = EPOCH )


epoch loss - 0.34370337426662445: 100%|██████████| 10/10 [43:58<00:00, 263.82s/it]


In [125]:

for i in range(EPOCH):
    bar_loader = tqdm(total = len(dataloader) ,)
    losses = []
    for i in dataloader:
        users, items, labels = i
        optimizer.zero_grad()
        logits = model(users, items)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels
        )
        loss.backward()
        optimizer.step()
        bar_loader.update(1)
        bar_loader.set_description(f'batch loss - {loss.item()}')
        losses.append(loss.item())
    
    bar.update(1)
    bar.set_description(f'epoch loss - {sum(losses)/len(losses)}')
    


batch loss - 0.35481756925582886: 100%|██████████| 2/2 [43:22<00:00, 1301.40s/it]




batch loss - 2.4728918075561523: 100%|██████████| 2/2 [00:04<00:00,  2.06s/it]




batch loss - 1.5181241035461426: 100%|██████████| 2/2 [00:03<00:00,  1.98s/it]




batch loss - 0.8410176634788513: 100%|██████████| 2/2 [00:03<00:00,  1.96s/it]




batch loss - 0.5034968256950378: 100%|██████████| 2/2 [00:04<00:00,  2.13s/it]




batch loss - 0.36193710565567017: 100%|██████████| 2/2 [00:04<00:00,  2.05s/it]

In [126]:
USER = 0

preds = list(model.pred_top_k(torch.tensor([USER]), 10)[1][0].numpy())
df[(df['user_index'] == USER) & (df['node_index'].isin(user2seen[USER]))]['names'].tolist()


['root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Кузов',
 'root -> Услуги -> Предложения услуг -> Красота, здоровье -> СПА-услуги, массаж',
 'root -> Услуги -> Предложения услуг -> Красота, здоровье -> СПА-услуги, массаж',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения']

In [127]:
[node2name[index2node[i]] for i in preds]

['root -> Электроника -> Телефоны -> Мобильные телефоны -> Apple',
 'root -> Транспорт -> Запчасти и аксессуары -> Шины, диски и колёса -> Легковые шины',
 'root -> Услуги -> Предложения услуг -> Красота, здоровье -> СПА-услуги, массаж',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения',
 'root -> Личные вещи -> Товары для детей и игрушки -> Детская мебель',
 'root -> Услуги -> Предложения услуг -> Автосервис, аренда -> Автосервис -> Диагностика и ремонт авто',
 'root -> Личные вещи -> Красота и здоровье -> Приборы и аксессуары',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Кузов',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Электрооборудование',
 'root -> Электроника -> Настольные компьютеры -> Системные блоки']

In [128]:
K = 100

test_users = df_test['user_index'].unique()


preds = model.pred_top_k(torch.tensor(test_users), K)[1].numpy()
df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, K)]for i in range(len(preds))]})

df_preds = df_preds.explode(['node_index', 'rank']).merge(
    df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
    on = ['user_index', 'node_index'],
    how='left' ,
)
df_preds['relevant'] = df_preds['relevant'].fillna(0)

In [129]:

def calc_hitrate(df_preds, K):
    return  df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].max().mean()

def calc_prec(df_preds, K):
    return  (df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].mean()).mean()
    
hitrate = calc_hitrate(df_preds, K)

hitrate, K

(0.8215892053973014, 100)

In [130]:
calc_prec(df_preds, 30)

0.04257871064467766

In [131]:
df_train['node_index'].max()

2175

In [132]:
top_popular = df_train[['node_index']].assign(v=1).groupby('node_index').count().reset_index().sort_values(by='v').tail(K)['node_index'].values


In [133]:
node2name[index2node[top_popular[-1]]]

'root -> Транспорт -> Автомобили -> Внедорожник'

In [134]:
df_preds_top_poplular = pd.DataFrame({'node_index': [list(top_popular) for i in test_users], 'user_index': test_users, 'rank': [[j for j in range(0, K)]for i in range(len(test_users))]})


df_preds_top_poplular = df_preds_top_poplular.explode(
    ['node_index', 'rank']
).merge(
    df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
    on = ['user_index', 'node_index'],
    how='left' ,
)
df_preds_top_poplular['relevant'] = df_preds_top_poplular['relevant'].fillna(0)

calc_hitrate(df_preds_top_poplular, K)

0.6836581709145427

In [136]:
prec = calc_prec(df_preds, 30)
prec

0.04257871064467766

In [138]:
hitrate = calc_hitrate(df_preds, 30)
hitrate

0.697151424287856

<!-- логируем бейзлайн -->

In [139]:
import mlflow

mlflow.set_tracking_uri('http://84.201.128.89:90/')
mlflow.set_experiment('homework-oafateev')

with mlflow.start_run(run_name='baseline'):
    mlflow.log_metrics(
        {
            'prec': prec, 
            'hitrate': hitrate,
        }
    )
    mlflow.log_params(
        {
        'model_name': 'LatentFactorModel',
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'EPOCH': EPOCH,
        'edim': EDIM,
        'NUM_NEGATIVES': NUM_NEGATIVES,
        'OPTIMIZER_NAME': 'Adam'
        }
    )

In [140]:
from sklearn.model_selection import ParameterGrid



param_grid = {
    'edim': [64, 128],
    'batch_size': [10000, 50000],
    'lr': [1, 0.1],
    'epoch': [5, 10],
    'num_negatives': [7, 5],
    'optimizer': ['Adam', 'RMSprop', 'Adagrad'] 
}
version = 0
for params in ParameterGrid(param_grid):
    model = LatentFactorModel(params['edim'], df_train['user_index'].values, df_train['node_index'].values)
    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr=params['lr'])

    for epoch in range(params['epoch']):
        for batch in dataloader:
            users, items, labels = batch
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)
            loss.backward()
            optimizer.step()

    preds = model.pred_top_k(torch.tensor(test_users), 30)[1].numpy()
    df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, 30)] for _ in range(len(preds))]})
    df_preds = df_preds.explode(['node_index', 'rank']).merge(
        df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
        on=['user_index', 'node_index'],
        how='left'
    )
    df_preds['relevant'] = df_preds['relevant'].fillna(0)
   

    hitrate = calc_hitrate(df_preds, K)
    prec = calc_prec(df_preds, 30)
    with mlflow.start_run(run_name=f"LatentFactorModel_{version}"):
        mlflow.log_params(params)
        mlflow.log_metrics(
            {
            'prec': prec, 
            'hitrate': hitrate,
            }
        )
    version += 1


In [141]:
from sklearn.model_selection import ParameterGrid


param_grid = {
    'edim': [64, 128],
    'batch_size': [10000, 50000],
    'lr': [1, 0.1],
    'epoch': [5, 10],
    'num_negatives': [7, 5],
    'optimizer': ['SGD','AdamW']
}
version = 97
for params in ParameterGrid(param_grid):
    model = LatentFactorModel(params['edim'], df_train['user_index'].values, df_train['node_index'].values)
    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr=params['lr'])

    for epoch in range(params['epoch']):
        for batch in dataloader:
            users, items, labels = batch
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)
            loss.backward()
            optimizer.step()

    preds = model.pred_top_k(torch.tensor(test_users), 30)[1].numpy()
    df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, 30)] for _ in range(len(preds))]})
    df_preds = df_preds.explode(['node_index', 'rank']).merge(
        df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
        on=['user_index', 'node_index'],
        how='left'
    )
    df_preds['relevant'] = df_preds['relevant'].fillna(0)

    hitrate = calc_hitrate(df_preds, K)
    prec = calc_prec(df_preds, 30)
    with mlflow.start_run(run_name=f"LatentFactorModel_{version}"):
        mlflow.log_params(params)
        mlflow.log_metrics(
            {
            'prec': prec, 
            'hitrate': hitrate,
            }
        )
    version += 1


In [142]:
from sklearn.model_selection import ParameterGrid


param_grid = {
    'edim': [64],
    'batch_size': [15000],
    'lr': [0.47],
    'epoch': [30],
    'num_negatives': [10],
    'optimizer': ['Adagrad']
}
version = 1000
for params in ParameterGrid(param_grid):
    model = LatentFactorModel(params['edim'], df_train['user_index'].values, df_train['node_index'].values)
    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr=params['lr'])

    for epoch in range(params['epoch']):
        for batch in dataloader:
            users, items, labels = batch
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)
            loss.backward()
            optimizer.step()

    preds = model.pred_top_k(torch.tensor(test_users), 30)[1].numpy()
    df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, 30)] for _ in range(len(preds))]})
    df_preds = df_preds.explode(['node_index', 'rank']).merge(
        df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
        on=['user_index', 'node_index'],
        how='left'
    )
    df_preds['relevant'] = df_preds['relevant'].fillna(0)

    hitrate = calc_hitrate(df_preds, K)
    prec = calc_prec(df_preds, 30)
    with mlflow.start_run(run_name=f"LatentFactorModel_{version}"):
        mlflow.log_params(params)
        mlflow.log_metrics(
            {
            'prec': prec, 
            'hitrate': hitrate,
            }
        )
    version += 1


In [143]:
from sklearn.model_selection import ParameterGrid


param_grid = {
    'edim': [64],
    'batch_size': [15000],
    'lr': [0.47],
    'epoch': [30],
    'num_negatives': [10],
    'optimizer': ['Adagrad']
}
version = 10100
for params in ParameterGrid(param_grid):
    model = LatentFactorModel(params['edim'], df_train['user_index'].values, df_train['node_index'].values)
    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr=params['lr'])

    for epoch in range(params['epoch']):
        for batch in dataloader:
            users, items, labels = batch
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)
            loss.backward()
            optimizer.step()

    preds = model.pred_top_k(torch.tensor(test_users), 30)[1].numpy()
    df_preds = pd.DataFrame({'node_index': list(preds), 'user_index': test_users, 'rank': [[j for j in range(0, 30)] for _ in range(len(preds))]})
    df_preds = df_preds.explode(['node_index', 'rank']).merge(
        df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
        on=['user_index', 'node_index'],
        how='left'
    )
    df_preds['relevant'] = df_preds['relevant'].fillna(0)

    hitrate = calc_hitrate(df_preds, K)
    prec = calc_prec(df_preds, 30)
    with mlflow.start_run(run_name=f"LatentFactorModel_{version}"):
        mlflow.log_params(params)
        mlflow.log_metrics(
            {
            'prec': prec, 
            'hitrate': hitrate,
            }
        )
    version += 1
